In [ ]:
!pip install flask pyngrok diffusers transformers accelerate safetensors flask-cors



In [ ]:
# Log in to Hugging Face (You need an account and an access token)
from huggingface_hub import login

# Paste your Hugging Face token here
login(token="Paste your Hugging Face token here")  # Get one from https://huggingface.co/settings/tokens

In [ ]:
!ngrok config add-authtoken 2yFPoiVEfrpouTXWYSAZxyAx6WJ_JYyEhRbf9oYfjZcwh645

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS for robustness
from pyngrok import ngrok
from diffusers import EulerDiscreteScheduler, StableDiffusionPipeline
import torch
import base64
import io  # Required for in-memory file handling

app = Flask(__name__)
CORS(app) # Enable CORS to prevent browser errors

# Load the model (same as before)
pipe = StableDiffusionPipeline.from_pretrained(
    "Lykon/dreamshaper-8",
    torch_dtype=torch.float16,
    scheduler=EulerDiscreteScheduler.from_pretrained("Lykon/dreamshaper-8", subfolder="scheduler"),
    safety_checker=None
).to("cuda")


@app.route("/generate", methods=["POST"])
def generate():
    data = request.json
    prompt = data.get("prompt", "")
    neg_prompt = data.get("negative_prompt", "ugly, blurry, deformed, low quality, bad anatomy, extra limbs, extra fingers, mutated, watermark, signature, text, jpeg artifacts, grainy, out of frame, cropped, nartfixer, nfixer, nrealfixer, easynegative, nsfw, porn, ((lens flare)), canvas frame, blurry, duplicate, extra arms, extra legs, fused fingers, long neck, morbid, out of frame, poorly drawn hands, poorly drawn face, ugly, poorly drawn, tiling, bad art, nude, BadDream, FastNegativeV2")
    guidance = float(data.get("guidance_scale", 7.5))
    aspect = data.get("aspect", "square")

    # Set width and height based on aspect ratio (same as before)
    if aspect == "portrait":
        width, height = 512, 768
    else:  # square
        width, height = 512, 512

    # Generate 2 images (same as before)
    images = pipe(
        prompt=[prompt] * 2,
        negative_prompt=[neg_prompt] * 2,
        guidance_scale=guidance,
        width=width,
        height=height,
        num_inference_steps=25
    ).images

    # --- NEW LOGIC: Encode images to Base64 ---
    base64_images = []
    for img in images:
        # Create an in-memory binary stream
        buffer = io.BytesIO()
        # Save the image to the stream in PNG format
        img.save(buffer, format="PNG")
        # Get the byte data from the stream
        img_bytes = buffer.getvalue()
        # Encode the bytes to a Base64 string
        encoded_string = base64.b64encode(img_bytes).decode("utf-8")
        base64_images.append(encoded_string)

    # Return a JSON object containing the list of Base64 strings
    return jsonify({
        "images": base64_images
    })

# The "/download" route is no longer needed and has been removed.

# Start server with ngrok (same as before)
public_url = ngrok.connect(5000)
print("🚀 Public URL:", public_url)
app.run(port=5000)
